# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [3]:
%nuclio config spec.build.baseImage = "mlrun/mlrun:0.4.5"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun:0.4.5'


In [4]:
%%nuclio cmd -c
python -m pip install -U kfserving 
python -m pip install numpy cloudpickle

In [5]:
import os
from cloudpickle import load

import kfserving

import numpy as np

from typing import List

In [6]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, classifier = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.classifier = classifier
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.

        HACK FOR NOW
        ============
        
        * need instructions on how to recreate whatever is in this 
        model_dir, 
        * currently we grab the first file that end with 'pkl' and 
        set it as model for inference. It could be a json file,
        onnx, etc...
        """
        if self.model_dir.endswith('.pkl'):
            model_file = self.model_dir
        else:
            for file in os.path.listdir(self.model_dir):
                if file.endswith('.pkl'):
                    model_file = os.pth.join(self.model_dir, file)
                    break
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [7]:
# nuclio: end-code